In [ ]:
from src.utils import compact_to_expanded, imputation, format_plot
from src.model_utils import plot_forecast, evaluate_performance, forecast_bias
import os
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.metrics import mean_squared_error as mse
from pmdarima import model_selection
from pmdarima import ARIMA as pm_ARIMA
from sklearn.model_selection import ParameterGrid
from statsforecast.models import (
    HoltWinters,
    AutoETS,
    ARIMA
)
import random
np.random.seed(42)
random.seed(42)
tqdm.pandas()

In [ ]:
%cd ../..
project_folder = Path("AI/london_smart_meters")
source_data = project_folder/"data"
preprocessed_data = project_folder/"output"/"preprocessed_data"
dst_img = project_folder/"output"/"img"
dst_result = project_folder/"output"/"result"

/home/hien/Work


In [ ]:
lclid_acorn_map = pd.read_pickle(preprocessed_data/"london_smart_meters_lclid_acorn_map.pkl")

In [4]:
print(lclid_acorn_map.columns)

Index(['LCLid', 'file', 'Acorn_grouped'], dtype='object')


In [5]:
print(lclid_acorn_map['Acorn_grouped'].unique())

['Affluent' 'ACORN-' 'Adversity' 'ACORN-U' 'Comfortable']


In [6]:
affluent_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Affluent", ["LCLid",'file']]
acorn_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="ACORN-", ["LCLid",'file']]
adversity_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Adversity", ["LCLid",'file']]
acornu_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="ACORN-U", ["LCLid",'file']]
comfortable_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Comfortable", ["LCLid",'file']]

In [7]:
print(len(acorn_households))
print(len(acornu_households))

2
49


In [8]:
selected_households = pd.concat(
    [
        affluent_households.sample(50, random_state=76),
        comfortable_households.sample(50, random_state=76),
        adversity_households.sample(50, random_state=76),
        acorn_households.sample(len(acorn_households), random_state=76),
        acornu_households.sample(len(acornu_households), random_state=76),
    ]
)
selected_households['block']=selected_households.file.str.split("_", expand=True).iloc[:,1].astype(int)


In [9]:
selected_households

,LCLid,file,block
897,MAC005300,block_14,14
1108,MAC005521,block_18,18
1049,MAC001979,block_17,17
1596,MAC005220,block_27,27
1313,MAC004650,block_21,21
...,...,...,...
757,MAC003652,block_111,111
740,MAC004515,block_110,110
700,MAC000099,block_110,110
708,MAC002152,block_110,110


In [ ]:
# extracting the paths to the different blocks and extracting the starting and ending blocks
path_blocks = [
    (p, *list(map(int, p.name.split("_")[5].split(".")[0].split("-"))))
    for p in preprocessed_data.glob(
        "london_smart_meters_merged_block*"
    )
]

In [11]:
household_df_l = []
for path, start_b, end_b in tqdm(path_blocks):
    block_df = pd.read_parquet(path)
    selected_households['block'].between
    mask = selected_households['block'].between(start_b, end_b)
    lclids = selected_households.loc[mask, "LCLid"]
    household_df_l.append(block_df.loc[block_df.LCLid.isin(lclids)])


100%|██████████| 14/14 [00:11<00:00,  1.22it/s]


In [12]:
block_df = pd.concat(household_df_l)
del household_df_l
block_df.head()

,LCLid,start_timestamp,frequency,energy_consumption,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
3199,MAC003069,2012-05-16,30min,"[0.069, 0.0819999999999999, 0.093, 0.074, 0.07...",31344,ToU,ACORN-F,Comfortable,block_56,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[284, 284, 285, 285, 277, 277, 277, 277, 277, ...","[6.33, 6.33, 5.19, 5.19, 4.7, 4.7, 3.89, 3.89,...","[3.08, 3.08, 2.64, 2.64, 2.93, 2.93, 2.93, 2.9...","[1023.94, 1023.94, 1024.23, 1024.23, 1024.33, ...","[4.23, 4.23, 2.84, 2.84, 2.27, 2.27, 1.41, 1.4...","[2.83, 2.83, 2.85, 2.85, 2.83, 2.83, 2.7, 2.7,...","[rain, rain, rain, rain, rain, rain, rain, rai...","[clear-night, clear-night, clear-night, clear-...","[0.8, 0.8, 0.84, 0.84, 0.88, 0.88, 0.93, 0.93,...","[Clear, Clear, Clear, Clear, Clear, Clear, Cle..."
3246,MAC002220,2012-06-30,30min,"[0.093, 0.077, 0.1009999999999999, 0.084, 0.03...",29184,Std,ACORN-G,Comfortable,block_57,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[205, 205, 202, 202, 204, 204, 199, 199, 198, ...","[16.19, 16.19, 16.13, 16.13, 15.68, 15.68, 15....","[13.18, 13.18, 13.28, 13.28, 13.36, 13.36, 13....","[1006.92, 1006.92, 1006.65, 1006.65, 1006.57, ...","[16.19, 16.19, 16.13, 16.13, 15.68, 15.68, 15....","[4.24, 4.24, 4.18, 4.18, 3.88, 3.88, 4.09, 4.0...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, par...","[0.82, 0.82, 0.83, 0.83, 0.86, 0.86, 0.9, 0.9,...","[Partly Cloudy, Partly Cloudy, Mostly Cloudy, ..."
3262,MAC004338,2012-03-16,30min,"[0.043, 0.034, 0.031, 0.038, 0.042, 0.04, 0.02...",34272,Std,ACORN-G,Comfortable,block_57,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[257, 257, 255, 255, 254, 254, 253, 253, 252, ...","[9.19, 9.19, 8.71, 8.71, 8.64, 8.64, 8.59, 8.5...","[5.71, 5.71, 5.08, 5.08, 5.33, 5.33, 5.01, 5.0...","[1022.39, 1022.39, 1022.22, 1022.22, 1021.82, ...","[7.16, 7.16, 6.39, 6.39, 6.33, 6.33, 6.48, 6.4...","[3.7, 3.7, 4.07, 4.07, 4.02, 4.02, 3.6, 3.6, 3...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, clo...","[0.79, 0.79, 0.78, 0.78, 0.8, 0.8, 0.78, 0.78,...","[Mostly Cloudy, Mostly Cloudy, Overcast, Overc..."
3280,MAC001807,2012-06-07,30min,"[0.047, 0.133, 0.032, 0.156, 0.034, 0.126, 0.0...",30288,Std,ACORN-G,Comfortable,block_58,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[213, 213, 195, 195, 182, 182, 167, 167, 164, ...","[12.61, 12.61, 12.6, 12.6, 12.98, 12.98, 12.99...","[11.1, 11.1, 11.14, 11.14, 11.65, 11.65, 12.09...","[1003.35, 1003.35, 1002.78, 1002.78, 1002.47, ...","[12.61, 12.61, 12.6, 12.6, 12.98, 12.98, 12.99...","[3.19, 3.19, 2.34, 2.34, 2.34, 2.34, 2.66, 2.6...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, cle...","[0.91, 0.91, 0.91, 0.91, 0.92, 0.92, 0.94, 0.9...","[Partly Cloudy, Partly Cloudy, Clear, Clear, P..."
3285,MAC001985,2012-06-13,30min,"[0.055, 0.048, 0.021, 0.015, 0.026, 0.00699999...",30000,Std,ACORN-G,Comfortable,block_58,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[349, 349, 9, 9, 2, 2, 25, 25, 20, 20, 31, 31,...","[10.07, 10.07, 9.13, 9.13, 8.4, 8.4, 7.75, 7.7...","[6.76, 6.76, 6.72, 6.72, 6.79, 6.79, 6.34, 6.3...","[1014.57, 1014.57, 1014.56, 1014.56, 1014.74, ...","[10.07, 10.07, 9.13, 9.13, 8.4, 8.4, 7.75, 7.7...","[0.84, 0.84, 1.15, 1.15, 0.59, 0.59, 0.79, 0.7...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, cle...","[0.8, 0.8, 0.85, 0.85, 0.9, 0.9, 0.91, 0.91, 0...","[Partly Cloudy, Partly Cloudy, Clear, Clear, C..."


In [13]:
#Converting to expanded form
exp_block_df = compact_to_expanded(block_df, timeseries_col = 'energy_consumption',
static_cols = ["frequency", "series_length", "stdorToU", "Acorn", "Acorn_grouped", "file"],
time_varying_cols = ['holidays', 'visibility', 'windBearing', 'temperature', 'dewPoint',
       'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon',
       'humidity', 'summary'],
ts_identifier = "LCLid")

exp_block_df.head()


100%|██████████| 185/185 [00:00<00:00, 352.77it/s]


,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-05-16 00:00:00,MAC003069,0.069,30min,31344,ToU,ACORN-F,Comfortable,block_56,NO_HOLIDAY,...,284,6.33,3.08,1023.94,4.23,2.83,rain,clear-night,0.80,Clear
1,2012-05-16 00:30:00,MAC003069,0.082,30min,31344,ToU,ACORN-F,Comfortable,block_56,NO_HOLIDAY,...,284,6.33,3.08,1023.94,4.23,2.83,rain,clear-night,0.80,Clear
2,2012-05-16 01:00:00,MAC003069,0.093,30min,31344,ToU,ACORN-F,Comfortable,block_56,NO_HOLIDAY,...,285,5.19,2.64,1024.23,2.84,2.85,rain,clear-night,0.84,Clear
3,2012-05-16 01:30:00,MAC003069,0.074,30min,31344,ToU,ACORN-F,Comfortable,block_56,NO_HOLIDAY,...,285,5.19,2.64,1024.23,2.84,2.85,rain,clear-night,0.84,Clear
4,2012-05-16 02:00:00,MAC003069,0.078,30min,31344,ToU,ACORN-F,Comfortable,block_56,NO_HOLIDAY,...,277,4.70,2.93,1024.33,2.27,2.83,rain,clear-night,0.88,Clear


# Train Test Valildation Split

In [14]:
test_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==2)
val_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==1)

train = exp_block_df[~(val_mask|test_mask)]
val = exp_block_df[val_mask]
test = exp_block_df[test_mask]
print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}")
print(f"Max Date in Train: {train.timestamp.max()} | Min Date in Validation: {val.timestamp.min()} | Min Date in Test: {test.timestamp.min()}")


# of Training samples: 5243376 | # of Validation samples: 275280 | # of Test samples: 239760
Max Date in Train: 2013-12-31 23:30:00 | Min Date in Validation: 2014-01-01 00:00:00 | Min Date in Test: 2014-02-01 00:00:00


In [15]:
train.head()

,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-05-16 00:00:00,MAC003069,0.069,30min,31344,ToU,ACORN-F,Comfortable,block_56,NO_HOLIDAY,...,284,6.33,3.08,1023.94,4.23,2.83,rain,clear-night,0.80,Clear
1,2012-05-16 00:30:00,MAC003069,0.082,30min,31344,ToU,ACORN-F,Comfortable,block_56,NO_HOLIDAY,...,284,6.33,3.08,1023.94,4.23,2.83,rain,clear-night,0.80,Clear
2,2012-05-16 01:00:00,MAC003069,0.093,30min,31344,ToU,ACORN-F,Comfortable,block_56,NO_HOLIDAY,...,285,5.19,2.64,1024.23,2.84,2.85,rain,clear-night,0.84,Clear
3,2012-05-16 01:30:00,MAC003069,0.074,30min,31344,ToU,ACORN-F,Comfortable,block_56,NO_HOLIDAY,...,285,5.19,2.64,1024.23,2.84,2.85,rain,clear-night,0.84,Clear
4,2012-05-16 02:00:00,MAC003069,0.078,30min,31344,ToU,ACORN-F,Comfortable,block_56,NO_HOLIDAY,...,277,4.70,2.93,1024.33,2.27,2.83,rain,clear-night,0.88,Clear


In [ ]:
train.to_parquet(preprocessed_data/"selected_blocks_train.parquet")
val.to_parquet(preprocessed_data/"selected_blocks_val.parquet")
test.to_parquet(preprocessed_data/"selected_blocks_test.parquet")

# Filling in missing values

In [17]:
train.isnull().sum()

timestamp                   0
LCLid                       0
energy_consumption     168038
frequency                   0
series_length               0
stdorToU                    0
Acorn                       0
Acorn_grouped               0
file                        0
holidays                    0
visibility                  0
windBearing                 0
temperature                 0
dewPoint                    0
pressure                 3844
apparentTemperature         0
windSpeed                   0
precipType                  0
icon                        0
humidity                    0
summary                     0
dtype: int64

In [18]:
test.isnull().sum()

timestamp                  0
LCLid                      0
energy_consumption     14928
frequency                  0
series_length              0
stdorToU                   0
Acorn                      0
Acorn_grouped              0
file                       0
holidays                   0
visibility                 0
windBearing                0
temperature                0
dewPoint                   0
pressure                   0
apparentTemperature        0
windSpeed                  0
precipType                 0
icon                       0
humidity                   0
summary                    0
dtype: int64

In [19]:
val.isnull().sum()

timestamp                  0
LCLid                      0
energy_consumption     19152
frequency                  0
series_length              0
stdorToU                   0
Acorn                      0
Acorn_grouped              0
file                       0
holidays                   0
visibility                 0
windBearing                0
temperature                0
dewPoint                   0
pressure                   0
apparentTemperature        0
windSpeed                  0
precipType                 0
icon                       0
humidity                   0
summary                    0
dtype: int64

In [20]:
ts_train = train.set_index("timestamp")
train_imputed = imputation(ts_train)
train_imputed = train_imputed.drop('energy_consumption', axis=1)

ts_val = val.set_index("timestamp")
val_imputed = imputation(ts_val)
val_imputed = val_imputed.drop('energy_consumption', axis=1)

ts_test = test.set_index("timestamp")
test_imputed = imputation(ts_test)
test_imputed = test_imputed.drop('energy_consumption', axis=1)

In [21]:
train_imputed = train_imputed.reset_index()
val_imputed = val_imputed.reset_index()
test_imputed = test_imputed.reset_index()

In [ ]:
train_imputed.to_parquet(preprocessed_data/"selected_blocks_train_missing_imputed.parquet")
val_imputed.to_parquet(preprocessed_data/"selected_blocks_val_missing_imputed.parquet")
test_imputed.to_parquet(preprocessed_data/"selected_blocks_test_missing_imputed.parquet")

# Baseline Forecast

In [23]:
# this makes it so that the outputs of the predict methods have the id as a column 
# instead of as the index
os.environ['NIXTLA_ID_AS_COL'] = '1'

In [ ]:
train_df = pd.read_parquet(preprocessed_data/"selected_blocks_train_missing_imputed.parquet")
train_df = train_df[['LCLid',"timestamp","energy_consumption_imputed","frequency"]]
val_df = pd.read_parquet(preprocessed_data/"selected_blocks_val_missing_imputed.parquet")
val_df = val_df[['LCLid',"timestamp","energy_consumption_imputed","frequency"]]
test_df = pd.read_parquet(preprocessed_data/"selected_blocks_test_missing_imputed.parquet")
test_df = test_df[['LCLid',"timestamp","energy_consumption_imputed","frequency"]]

In [25]:
train_df

,LCLid,timestamp,energy_consumption_imputed,frequency
0,MAC003069,2012-05-16 00:00:00,0.069,30min
1,MAC003069,2012-05-16 00:30:00,0.082,30min
2,MAC003069,2012-05-16 01:00:00,0.093,30min
3,MAC003069,2012-05-16 01:30:00,0.074,30min
4,MAC003069,2012-05-16 02:00:00,0.078,30min
...,...,...,...,...
5243371,MAC002745,2013-12-31 21:30:00,0.506,30min
5243372,MAC002745,2013-12-31 22:00:00,0.437,30min
5243373,MAC002745,2013-12-31 22:30:00,0.503,30min
5243374,MAC002745,2013-12-31 23:00:00,0.434,30min


In [26]:
print("Min train_df Date: " , train_df.timestamp.min())
print("Max train_df Date: " , train_df.timestamp.max())
print("Min val_df Date: " , val_df.timestamp.min())
print("Max val_df Date: " , val_df.timestamp.max())
print("Min test_df Date: " , test_df.timestamp.min())
print("Max test_df Date: " , test_df.timestamp.max())

Min train_df Date:  2012-01-01 00:00:00
Max train_df Date:  2013-12-31 23:30:00
Min val_df Date:  2014-01-01 00:00:00
Max val_df Date:  2014-01-31 23:30:00
Min test_df Date:  2014-02-01 00:00:00
Max test_df Date:  2014-02-27 23:30:00


In [27]:
train_df = train_df[train_df.timestamp >'2012-01-01']
print("Min train_df Date: " , train_df.timestamp.min())


Min train_df Date:  2012-01-01 00:30:00


In [28]:
#picking a single time series from the dataset for illustration
freq = train_df.iloc[0]['frequency']
ts_train = train_df.loc[train_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption_imputed"]]
ts_val = val_df.loc[val_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption_imputed"]]
ts_test = test_df.loc[test_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption_imputed"]]


In [29]:
pred_df = pd.concat([ts_train, ts_val])

In [30]:
metrics = pd.DataFrame()

## Holt-Winters

In [31]:
results, metrics = (
    evaluate_performance(
        ts_train,
        ts_val,
        models = [ HoltWinters(error_type = 'A', season_length = 48)],
        metrics = [mae, mse, rmse, forecast_bias],
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption_imputed',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [32]:
metrics

,mean_absolute_error,mean_squared_error,root_mean_squared_error,forecast_bias,LCLid,Time Elapsed,Model
0,0.191257,0.100946,0.31772,10.48171,MAC000193,4.029585,HoltWinters


In [ ]:
model_name = ['HoltWinters']
model_display_name = ['HoltWinters']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig)
fig.update_layout(
    title=f"{model_name[0]}: "\
          f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mean_absolute_error']].iloc[0].item():.4f} | "\
          f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mean_squared_error']].iloc[0].item():.4f} | "\
          f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}"
)
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.write_image(dst_img/"holt-winter.png")
fig.show()

## ARIMA

In [34]:
results, metrics = (
    evaluate_performance(
        ts_train,
        ts_val,
        models = [ARIMA(order = (2,1,1), seasonal_order = (1,1,1), season_length = 48)],
        metrics = [mae, mse, rmse, forecast_bias],
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption_imputed',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [35]:
metrics

,mean_absolute_error,mean_squared_error,root_mean_squared_error,forecast_bias,LCLid,Time Elapsed,Model
0,0.191257,0.100946,0.317720,10.481710,MAC000193,4.029585,HoltWinters
1,0.203665,0.106539,0.326402,25.416513,MAC000193,20.193931,ARIMA


In [ ]:
## ARIMA
model_name = ['ARIMA']
model_display_name = ['ARIMA']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig)
fig.update_layout(
    title=f"{model_name[0]}: "\
          f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mean_absolute_error']].iloc[0].item():.4f} | "\
          f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mean_squared_error']].iloc[0].item():.4f} | "\
          f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}"
)
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.write_image(dst_img/"arima.png")
fig.show()

# Parameter Tuning

In [37]:
# # Defining the parameters
# param_grid = {'p': [1,2],
#               'd': [0,1],
#               'q': [1,2],
#               'P': [1,2],
#               'D': [0],
#               'Q': [1]}
# grid = ParameterGrid(param_grid)
# len(list(grid))

In [38]:
# # Initialize an empty list to store RMSE values for each parameter set
# rmse_list = []
# pred_df = pd.concat([ts_train, ts_val])
# # First, convert the timestamp column to datetime format
# pred_df['timestamp'] = pd.to_datetime(pred_df['timestamp'])
# # Set the timestamp column as the index
# pred_df = pred_df.set_index('timestamp')
# pred_df = pred_df.drop(['LCLid'], axis=1)

# # Iterate over each set of parameters in the grid
# for params in grid:

#   # Build an ARIMA model with the current set of parameters
#   model = pm_ARIMA(order=(params['p'], params['d'], params['q']),
#                 seasonal_order=(params['P'], params['D'], params['Q'], 48))

#   # Define the Rolling Forecast Cross-Validation strategy
#   cv = model_selection.RollingForecastCV(h=60,
#                                          step=30,
#                                          initial=pred_df.shape[0] - 360)

#   # Perform cross-validation scoring with the ARIMA model
#   cv_score = model_selection.cross_val_score(model,
#                                              y=pred_df['energy_consumption_imputed'],  # Target variable 'y'
#                                              scoring='mean_squared_error',  # Evaluation metric: Mean Squared Error
#                                              cv=cv,  # Cross-validation strategy
#                                              verbose=1,  # Verbosity level
#                                              error_score=10000000000000000000000  # Value to assign if fitting error occurs
#                                              )

#   # Calculate RMSE and store the error
#   rmse = np.sqrt(np.average(cv_score))
#   rmse_list.append(rmse)

In [39]:
# # Create a DataFrame to store the tuning results with parameters and corresponding RMSE values
# tuning_results = pd.DataFrame(grid)

# # Add the RMSE values calculated during parameter tuning to the DataFrame
# tuning_results['rmse'] = rmse_list

# tuning_results

In [40]:
# # Save the best parameters
# best_params = tuning_results[tuning_results['rmse'] == tuning_results['rmse'].min()].transpose()
# best_params

In [41]:
# # Fetch the best parameters
# p = int(best_params.loc['p'].iloc[0])
# d = int(best_params.loc['d'].iloc[0])
# q = int(best_params.loc['q'].iloc[0])
# P = int(best_params.loc['P'].iloc[0])
# D = int(best_params.loc['D'].iloc[0])
# Q = int(best_params.loc['Q'].iloc[0])

## AutoETS

In [42]:
results, metrics = (
    evaluate_performance(
        ts_train,
        ts_val,
        models = [ AutoETS(model = 'AAA',season_length = 48)],
        metrics = [mae, mse, rmse, forecast_bias],
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption_imputed',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
## AutoETS
model_name = ['AutoETS']
model_display_name = ['AutoETS']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig)
fig.update_layout(
    title=f"{model_name[0]}: "\
          f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mean_absolute_error']].iloc[0].item():.4f} | "\
          f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mean_squared_error']].iloc[0].item():.4f} | "\
          f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}"
)
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.write_image(dst_img/"autoets.png")
fig.show()

In [44]:
metrics

,mean_absolute_error,mean_squared_error,root_mean_squared_error,forecast_bias,LCLid,Time Elapsed,Model
0,0.191257,0.100946,0.317720,10.481710,MAC000193,4.029585,HoltWinters
1,0.203665,0.106539,0.326402,25.416513,MAC000193,20.193931,ARIMA
2,0.191257,0.100946,0.317720,10.481710,MAC000193,3.856325,AutoETS


## Validation

In [45]:
ids = list(train_df.LCLid.unique()[:500]) # slicing dataframe for the sake of time.  May want to consider setting this low if working on a slower machine

train_df = train_df[train_df.LCLid.isin(ids)]
val_df = val_df[val_df.LCLid.isin(ids)]
test_df = test_df[test_df.LCLid.isin(ids)]

print("Length of validation Data: ", len(val_df[val_df.LCLid =='MAC000948'].LCLid))

Length of validation Data:  1488


In [46]:
validation_models =  [AutoETS(model = 'AAA',season_length = 48),
                      ARIMA(order = (2,1,1), seasonal_order = (1,1,1), season_length = 48)]

validation_models_names = [model.__class__.__name__ for model in validation_models]
metric_df = pd.DataFrame([])
h_val = len(val_df[val_df.LCLid =='MAC000948'].LCLid)

In [47]:
aggval_metrics = pd.DataFrame()

baseline_val_pred_df, aggval_metrics = (
    evaluate_performance(
        train_df[["LCLid","timestamp","energy_consumption_imputed"]],
        val_df[["LCLid","timestamp","energy_consumption_imputed"]],
        models =validation_models,
        metrics = [mae, mse, rmse, forecast_bias],
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption_imputed',
        h = h_val,
        metric_df = aggval_metrics
        )
)

/home/hien/Work/AI/london_smart_meters/src/model_utils.py:171: RuntimeWarning:

divide by zero encountered in scalar divide

/home/hien/Work/AI/london_smart_meters/.venv/lib/python3.12/site-packages/statsforecast/arima.py:701: RuntimeWarning:

invalid value encountered in sqrt

/home/hien/Work/AI/london_smart_meters/src/model_utils.py:171: RuntimeWarning:

divide by zero encountered in scalar divide



In [48]:
aggval_metrics.head()

,mean_absolute_error,mean_squared_error,root_mean_squared_error,forecast_bias,LCLid,Time Elapsed,Model
0,0.115688,0.020533,0.143292,-61.554792,MAC003069,327.685274,AutoETS
1,0.043875,0.003487,0.059054,-16.991496,MAC002220,327.685274,AutoETS
2,0.115360,0.056887,0.238509,11.801314,MAC004338,327.685274,AutoETS
3,0.061357,0.010357,0.101768,-12.532731,MAC001807,327.685274,AutoETS
4,0.088098,0.027629,0.166221,20.875673,MAC001985,327.685274,AutoETS


In [49]:
baseline_val_pred_df[baseline_val_pred_df.LCLid =='MAC000173']

,LCLid,timestamp,energy_consumption_imputed,AutoETS,ARIMA
92256,MAC000173,2014-01-01 00:00:00,0.137,0.100279,0.256487
92257,MAC000173,2014-01-01 00:30:00,0.092,0.087512,0.164007
92258,MAC000173,2014-01-01 01:00:00,0.106,0.079246,0.109702
92259,MAC000173,2014-01-01 01:30:00,0.138,0.076074,0.089604
92260,MAC000173,2014-01-01 02:00:00,0.045,0.075061,0.080950
...,...,...,...,...,...
93739,MAC000173,2014-01-31 21:30:00,0.255,0.265651,0.301897
93740,MAC000173,2014-01-31 22:00:00,0.271,0.255902,0.301395
93741,MAC000173,2014-01-31 22:30:00,0.597,0.215468,0.267211
93742,MAC000173,2014-01-31 23:00:00,0.202,0.143761,0.187872


In [50]:
aggval_metrics[aggval_metrics.Model =='AutoETS'].sort_values(by='mean_squared_error', ascending = False)

,mean_absolute_error,mean_squared_error,root_mean_squared_error,forecast_bias,LCLid,Time Elapsed,Model
55,0.879341,1.114789e+00,1.055836,65.130260,MAC000321,327.685274,AutoETS
58,0.507991,6.361264e-01,0.797575,66.344194,MAC001979,327.685274,AutoETS
64,0.371269,5.036701e-01,0.709697,74.825606,MAC004428,327.685274,AutoETS
47,0.454546,3.490520e-01,0.590806,3.935768,MAC005220,327.685274,AutoETS
52,0.322427,3.370847e-01,0.580590,78.689882,MAC005008,327.685274,AutoETS
...,...,...,...,...,...,...,...
131,0.034418,1.989468e-03,0.044603,13.284201,MAC001074,327.685274,AutoETS
179,0.034732,1.907039e-03,0.043670,13.571226,MAC001288,327.685274,AutoETS
158,0.029000,1.579213e-03,0.039739,-1.751551,MAC000269,327.685274,AutoETS
108,0.025167,1.281279e-03,0.035795,-49.762293,MAC004142,327.685274,AutoETS


In [51]:
autoets_val_metric_df = aggval_metrics[aggval_metrics.Model =='AutoETS']
overall_metrics_val_autoets = {
    "Algorithm": "AutoETS",
    "MSE": mse(baseline_val_pred_df.energy_consumption_imputed.values, baseline_val_pred_df.AutoETS.values),
    "RMSE": rmse(baseline_val_pred_df.energy_consumption_imputed.values, baseline_val_pred_df.AutoETS.values),
    "MAE": mae(baseline_val_pred_df.energy_consumption_imputed.values, baseline_val_pred_df.AutoETS.values),
    "Forecast Bias": forecast_bias(baseline_val_pred_df.energy_consumption_imputed.values, baseline_val_pred_df.AutoETS.values)
}
overall_metrics_val_autoets

{'Algorithm': 'AutoETS',
 'MSE': 0.05760067669270902,
 'RMSE': 0.24000140977233658,
 'MAE': 0.12302987768895093,
 'Forecast Bias': 3.14997289851879}

In [52]:
arima_val_metric_df = aggval_metrics[aggval_metrics.Model =='ARIMA']
overall_metrics_val_arima = {
    "Algorithm": "ARIMA",
    "MSE": mse(baseline_val_pred_df.energy_consumption_imputed.values, baseline_val_pred_df.ARIMA.values),
    "RMSE": rmse(baseline_val_pred_df.energy_consumption_imputed.values, baseline_val_pred_df.ARIMA.values),
    "MAE": mae(baseline_val_pred_df.energy_consumption_imputed.values, baseline_val_pred_df.ARIMA.values),
    "Forecast Bias": forecast_bias(baseline_val_pred_df.energy_consumption_imputed.values, baseline_val_pred_df.ARIMA.values)
}
overall_metrics_val_arima

{'Algorithm': 'ARIMA',
 'MSE': 0.08410813308710255,
 'RMSE': 0.2900140222249651,
 'MAE': 0.1506598273968745,
 'Forecast Bias': 5.198977467820843}

In [53]:
baseline_val_metrics_df = pd.DataFrame([overall_metrics_val_autoets, overall_metrics_val_arima])
baseline_val_metrics_df

,Algorithm,MSE,RMSE,MAE,Forecast Bias
0,AutoETS,0.057601,0.240001,0.12303,3.149973
1,ARIMA,0.084108,0.290014,0.15066,5.198977


In [ ]:
baseline_val_pred_df.to_pickle(dst_result/"baseline_val_prediction_df.pkl")
baseline_val_metrics_df.to_pickle(dst_result/"baseline_val_metrics_df.pkl")
aggval_metrics.to_pickle(dst_result/"baseline_val_aggregate_metrics.pkl")

## Test

In [78]:
# Calculate the forecast horizon for test data
h_test = len(test_df[test_df.LCLid == 'MAC000948'].LCLid)

# Create a combined dataset of train + validation data for final model training
train_val_df = pd.concat([
    train_df[["LCLid", "timestamp", "energy_consumption_imputed"]],
    val_df[["LCLid", "timestamp", "energy_consumption_imputed"]]
])

# Initialize metrics DataFrame for test results
aggtest_metrics = pd.DataFrame()

# Apply models to test data
baseline_test_pred_df, aggtest_metrics = (
    evaluate_performance(
        train_val_df,  # Use combined train+validation data for final model training
        test_df[["LCLid", "timestamp", "energy_consumption_imputed"]],  # Test on test data
        models=validation_models,  # Use the same models
        metrics=[mae, mse, rmse, forecast_bias],
        freq=freq,
        level=[],
        id_col='LCLid',
        time_col='timestamp',
        target_col='energy_consumption_imputed',
        h=h_test,
        metric_df=aggtest_metrics
    )
)

/home/hien/Work/AI/london_smart_meters/src/model_utils.py:171: RuntimeWarning:

divide by zero encountered in scalar divide

/home/hien/Work/AI/london_smart_meters/src/model_utils.py:171: RuntimeWarning:

divide by zero encountered in scalar divide



In [79]:
aggtest_metrics.head()

,mean_absolute_error,mean_squared_error,root_mean_squared_error,forecast_bias,LCLid,Time Elapsed,Model
0,0.063103,0.014058,0.118568,13.096588,MAC003069,322.646377,AutoETS
1,0.031380,0.002216,0.047077,-17.411985,MAC002220,322.646377,AutoETS
2,0.118270,0.054404,0.233246,-11.551055,MAC004338,322.646377,AutoETS
3,0.049294,0.009882,0.099409,-1.060129,MAC001807,322.646377,AutoETS
4,0.088535,0.028487,0.168780,22.990714,MAC001985,322.646377,AutoETS


In [80]:
autoets_test_metric_df = aggtest_metrics[aggtest_metrics.Model =='AutoETS']
overall_metrics_test_autoets = {
    "Algorithm": "AutoETS",
    "MSE": mse(baseline_test_pred_df.energy_consumption_imputed.values, baseline_test_pred_df.AutoETS.values),
    "RMSE": rmse(baseline_test_pred_df.energy_consumption_imputed.values, baseline_test_pred_df.AutoETS.values),
    "MAE": mae(baseline_test_pred_df.energy_consumption_imputed.values, baseline_test_pred_df.AutoETS.values),
    "Forecast Bias": forecast_bias(baseline_test_pred_df.energy_consumption_imputed.values, baseline_test_pred_df.AutoETS.values)
}
overall_metrics_test_autoets

{'Algorithm': 'AutoETS',
 'MSE': 0.05867107622735972,
 'RMSE': 0.24222113084402797,
 'MAE': 0.11912738798844133,
 'Forecast Bias': -10.707988370940951}

In [82]:
arima_test_metric_df = aggtest_metrics[aggtest_metrics.Model =='ARIMA']
overall_metrics_test_arima = {
    "Algorithm": "ARIMA",
    "MSE": mse(baseline_test_pred_df.energy_consumption_imputed.values, baseline_test_pred_df.ARIMA.values),
    "RMSE": rmse(baseline_test_pred_df.energy_consumption_imputed.values, baseline_test_pred_df.ARIMA.values),
    "MAE": mae(baseline_test_pred_df.energy_consumption_imputed.values, baseline_test_pred_df.ARIMA.values),
    "Forecast Bias": forecast_bias(baseline_test_pred_df.energy_consumption_imputed.values, baseline_test_pred_df.ARIMA.values)
}
overall_metrics_test_arima

{'Algorithm': 'ARIMA',
 'MSE': 0.06942792983530538,
 'RMSE': 0.2634918022165118,
 'MAE': 0.13278325655313433,
 'Forecast Bias': -13.314742507049132}

In [83]:
baseline_test_metrics_df = pd.DataFrame([overall_metrics_test_autoets, overall_metrics_test_arima])
baseline_test_metrics_df

,Algorithm,MSE,RMSE,MAE,Forecast Bias
0,AutoETS,0.058671,0.242221,0.119127,-10.707988
1,ARIMA,0.069428,0.263492,0.132783,-13.314743


## Visualization

In [ ]:
fig = px.histogram(aggtest_metrics,
                   x="mean_absolute_error",
                   color="Model",
                   pattern_shape="Model",
                   marginal="box",
                   nbins=500,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mae", ylabel="Probability Density")
fig.update_layout(title="Distribution of MAE in the dataset", xaxis_range=[0,1])
fig.write_image(dst_img/"mae_dist.png")
fig.show()


In [ ]:
fig = px.histogram(aggtest_metrics,
                   x="mean_squared_error",
                   color="Model",
                   pattern_shape="Model",
                   marginal="box",
                   nbins=500,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mse", ylabel="Probability Density")
fig.update_layout(title="Distribution of MSE in the dataset", xaxis_range=[0,1.5])
fig.write_image(dst_img/"mse_dist.png")
fig.show()

In [ ]:
fig = px.histogram(aggtest_metrics,
                   x="root_mean_squared_error",
                   color="Model",
                   pattern_shape="Model",
                   marginal="box",
                   nbins=500,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="rmse", ylabel="Probability Density")
fig.update_layout(title="Distribution of RMSE in the dataset", xaxis_range=[0,1])
fig.write_image(dst_img/"rmse_dist.png")
fig.show()

In [ ]:
fig = px.histogram(aggtest_metrics,
                   x="forecast_bias",
                   color="Model",
                   pattern_shape="Model",
                   marginal="box",
                   nbins=250,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="forecast_bias", ylabel="Probability Density")
fig.update_layout(title="Distribution of BIAS in the dataset", xaxis_range=[-200,200])
fig.write_image(dst_img/"bias_dist.png")
fig.show()

In [64]:
display_lclid = "MAC003069"
ts_baseline_test_pred_df = baseline_test_pred_df.loc[baseline_test_pred_df.LCLid==display_lclid]
ts_aggtest_metrics = aggtest_metrics.loc[aggtest_metrics.LCLid==display_lclid]

In [ ]:
## AutoETS
model_name = ['AutoETS']
model_display_name = ['AutoETS']

fig = plot_forecast(ts_baseline_test_pred_df, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig)
fig.update_layout(
    title=f"{model_name[0]}: "\
          f"MSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Model==model_name[0]][['mean_squared_error']].iloc[0].item():.4f} | "\
          f"RMSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Model==model_name[0]][['root_mean_squared_error']].iloc[0].item():.4f} | "\
          f"MAE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Model==model_name[0]][['mean_absolute_error']].iloc[0].item():.4f} | "\
          f"BIAS: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}"
)
fig.update_xaxes(type="date", range=["2014-02-01", "2014-02-08"])
fig.write_image(dst_img/"autoets.png")
fig.show()

In [ ]:
## ARIMA
model_name = ['ARIMA']
model_display_name = ['ARIMA']

fig = plot_forecast(ts_baseline_test_pred_df, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig)
fig.update_layout(
    title=f"{model_name[0]}: "\
          f"MSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Model==model_name[0]][['mean_squared_error']].iloc[0].item():.4f} | "\
          f"RMSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Model==model_name[0]][['root_mean_squared_error']].iloc[0].item():.4f} | "\
          f"MAE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Model==model_name[0]][['mean_absolute_error']].iloc[0].item():.4f} | "\
          f"BIAS: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}"
)
fig.update_xaxes(type="date", range=["2014-02-01", "2014-02-08"])
fig.write_image(dst_img/"arima.png")
fig.show()

In [ ]:
baseline_test_pred_df.to_pickle(dst_result/"baseline_test_prediction_df.pkl")
baseline_test_metrics_df.to_pickle(dst_result/"baseline_test_metrics_df.pkl")
aggtest_metrics.to_pickle(dst_result/"baseline_test_aggregate_metrics.pkl")